In [217]:
pip install galois

In [218]:
# Needed libraries
import secrets
import math
import numpy as np
import hashlib
import galois

# Generate private seed

In [219]:
private_seed = secrets.token_bytes(32)
print(private_seed)

b'qr\\&o\xad \xa6\xb8%:z@W\r\xe7\x84?\x14\x91\xba_\xde\xa3m\xe2\xf5NPz\x90\x1d'


# SHAKE A.K.A function H (Squeeze public_seed and T)

In [220]:
v = 197
m = 57
lvl = 1

## SHAKE using hashlib

In [221]:
if(lvl==1):
  h_shake = hashlib.shake_128(private_seed)
else:
  h_shake = hashlib.shake_256(private_seed)
private_sponge = h_shake.digest(32+math.ceil(m/8)*v)
print(private_sponge)

b'\xf5kk\x85\x81\xabFa\xde\xc9\x88_\xf7\xa2@k\xf7*\xf7\xa8%\x96\xd9E\xb6\xae,\xb2b\xdd\xb7\xa2y\x17\xa7\xa9F\xc9"\xa7.\xab|\x80\xb9=\xe1hN"\xa6\xdf\x1b\xfdj\xf0W\xe4\x8d=\x06\xca\xeal_\x8ag\x9b\xaen\x199\xe9\xd8#\xfa\x04t\xa0\x96\xfeZ\xfb\xb0\xd1\x97\xcbF\xa5jYt\xe2\xb3\xb9\xf2\xcc\xa0\x9fm\xa5^\x9c\xe9|c\xd9[g\x14/\xba\xf4\xa4R?\x10\x10e\xe6#}\x1b\xaf\xf1"R\xcf\xa9CJ\xf4\x8eV{\x87s\x8f\xe6>\x16|yKedB\x99 \xd3\xb3H\x85\xb9q\x9c\xdd\xea\xe0\xbd\xb0\x9d\xd6\xd8,\x8c\xc5\x10\x05\x88\xeb\x88\x04}o\xf0\xd5\x92\xb2\xbf\x00\x92\xe9N\x0fD/\xfd\xf1\x08@\xaf-Gv\xfcB\x07\x15+(.8\xd8f\x8c\xbf\xec\xb5\x86\xedt\xc5\xa4\xfb\\\xfc^r]\xda5+\x1a\x1c\x1b\xbe\xdf\xe7\xfd\x06\x0c\x83g\xeevT\xb8\xcf\x07^\xc9\xe4\x1f\xe2\x1e\xff\xcd\x9eB\x10\xf6G\xc9\x111\xc8\x9b\x16G-\xaf\x92@\xdd\xb2\xcf\x1f\x05\xad\xc4Z\xf7\xa40\xa7\xa2\xce\x9c\xa1E\xb0\x02\xd41f\xf7%\x92_3\xf8\x8f\xf4@N\xedep\xcc\xb4H\x17XZd\xc5R\xc4x\xf4\x96\xe6\xef}\x1a\xdf\x8bp?\x01\r\xb1\xe7\xb9J\x81\x1a9y\xe6\xa8?j\xc4v\xcb\xf0\xdbp\xf7\x857&\x00q\x

#Extract public seed

In [222]:
public_seed = private_sponge[:32]
print(public_seed)

b'\xf5kk\x85\x81\xabFa\xde\xc9\x88_\xf7\xa2@k\xf7*\xf7\xa8%\x96\xd9E\xb6\xae,\xb2b\xdd\xb7\xa2'


#Extract T

In [223]:
def int8_to_bits(intByte:int)->list:
  """Get the first 8 bits of an integer as an array.

  The function takes one integer byte, extracts it's bits
  and returns an array containing the bits starting from
  the most significant bit.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  list
      An array with the first 8 bits of the integer
      starting from the most significant one.
  """
  storage = []
  for i in range(8):
    storage.insert(0,(intByte>>i)&1)
  return storage

In [224]:
def tMatrixRowGen(byteString:bytes,neededBits:int)->list:
  """Generate one row of the T matrix.

  The function takes one byte-string and extracts the specified
  ammount of bits (neededBits) to fill one row. When the ammount
  of bits doesn't fit completely in a byte, the most significant
  bits of the last byte are ignored.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the row.
  neededBits : int
    The ammount of bits a row needs to have.

  Returns
  -------
  list
      An array representing one row of the T matrix
  """
  row = []
  while(neededBits>8):
    row+=int8_to_bits(byteString[0])
    byteString = byteString[1:]
    neededBits = neededBits-8
  if(neededBits>0):
    row+=int8_to_bits(byteString[0])[-neededBits:]
  return row

def tMatrixGenerator(byteString:bytes,v:int,m:int)->np.ndarray:
  """Generate the T matrix.

  The function takes one byte-string and generates v rows
  each one with m bits extracted sequentially from a byteString
  using the tMatrixRowGen() function.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.
  v : int
    The ammount rows of the matrix.
  m : int
    The ammount columns of the matrix.

  Returns
  -------
  list
      An array representing the T matrix
  """
  mdivided8 =math.ceil(m/8)
  matrix = []
  for i in range(v):
    matrix.append(tMatrixRowGen(byteString,m))
    byteString = byteString[mdivided8:]
  return np.array(matrix)

In [225]:
T_base = private_sponge[32:]
T = tMatrixGenerator(T_base,v,m)
print(T)

[[0 1 1 ... 1 0 1]
 [0 0 1 ... 0 1 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 0 1 ... 0 1 1]
 [1 1 0 ... 0 1 0]
 [0 0 1 ... 1 0 0]]


# SHAKE (squeeze public map)

In [226]:
def subMatrixGenerator(byteString:bytes)->np.ndarray:
  """Generate a part of the L or Q1 matrix.

  The function takes one byte-string and returns one 16 row part
  of the L or Q1 matrix. It must be noted that the rows are generated as columns
  at first and then are transposed.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.

  Returns
  -------
  np.ndarray
      A matrix
  """
  matrix = []
  while(len(byteString)>0):
    matrix.append(int8_to_bits(byteString[0])+int8_to_bits(byteString[1]))
    byteString = byteString[2:]
  return np.array(matrix).T

In [227]:
n = m+v
g_equation = 2+2*n+v*(v+1)+2*v*m
mdivided16 = math.ceil(m/16)

#first g function iteration
g_shake = hashlib.shake_128(public_seed+bytes.fromhex('00'))
public_sponge = g_shake.digest(g_equation)
C_base=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
public_sponge = public_sponge[2:]
# L and Q1 have more than one dimension, so they are initilized
# differently
L_base = subMatrixGenerator(public_sponge[:2*n])
public_sponge = public_sponge[2*n:]
Q1_base = subMatrixGenerator(public_sponge)

#following g function iterations
for i in range(mdivided16):
  g_shake = hashlib.shake_128(public_seed+bytes.fromhex(f'0{i}'))
  public_sponge = g_shake.digest(g_equation)
  C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
  public_sponge = public_sponge[2:]
  L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
  public_sponge = public_sponge[2*n:]
  Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))

### Obtaining C

In [228]:
# we take the m bits (starting from the last generated bit) we need from
# the total of bits generated
C = C_base[-m:]
C = np.array(C)
print(C)

[1 0 0 0 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0
 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0]


### Obtaining L

In [229]:
#We discard the upper rows to get the specified dimensions
L = L_base[-m:]
print(L)

[[1 1 0 ... 0 1 0]
 [0 1 0 ... 0 1 0]
 [0 0 1 ... 0 1 1]
 ...
 [1 0 0 ... 0 1 1]
 [1 1 1 ... 0 1 0]
 [0 0 1 ... 1 1 0]]


### Obtaining $$Q_{1}$$

In [230]:
#we discard the upper row to get the specified dimensions
Q1=Q1_base[-m:]
print(Q1)

[[1 0 0 ... 0 1 0]
 [1 1 1 ... 0 1 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 0 0 ... 1 0 1]
 [0 0 0 ... 1 1 1]
 [0 1 1 ... 0 0 0]]


# find $$Q_2$$

In [231]:
def findPk1(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk1 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-v matrix representing the part of pk that is
      quadratic in the vinegar variables.
  """
  Pk1 = np.zeros([v,v],dtype=np.int8)
  column = 0
  for i in range(v):
    for j in range(i,v):
      Pk1[i,j]= q1[k,column]
      column+=1
    column+=m
  return Pk1

In [232]:
def findPk2(k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk2 algorithm of the LUOV specification.

  Parameters
  ----------
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-m matrix representing the part of pk that is bilinear in the
      vinegar variables and the oil variables.
  """
  Pk2 = np.zeros([v,m],dtype=np.int8)
  column = 0
  for i in range(v):
    column += v-i
    for j in range(m):
      Pk2[i,j]= q1[k,column]
      column+=1
  return Pk2

In [233]:
def findQ2(Q1:np.ndarray,T:np.ndarray)->galois.GF2:
  """An implementation of the findQ2 algorithm of the LUOV specification.

  Parameters
  ----------
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.
  t : np.ndarray
    A v-by-m matrix.

  Returns
  -------
  np.ndarray
      The second part of Macaulay matrix for quadratic part of P.
  """
  # we use galois to prevent the results of the operations from
  # being out of the finite field
  GF = galois.GF(2)
  T = GF(T)

  Q2 = GF(np.zeros([m,int(m*(m+1)/2)],dtype=np.int8))
  for k in range(m):
    Pk1 = GF(findPk1(k,Q1))
    Pk2 = GF(findPk2(k,Q1))
    Pk3 = -np.transpose(T)@Pk1@T+np.transpose(T)@Pk2
    column = 0
    for i in range(m):
      Q2[k,column]= Pk3[i,i]
      column += 1
      for j in range(i+1,m):
        Q2[k,column] = Pk3[i,j] + Pk3[j,i]
        column+=1
  return Q2

In [234]:
Q2 = findQ2(Q1,T)
print(Q2)

[[0 1 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 1 ... 0 1 0]
 [1 1 1 ... 1 1 0]
 [1 1 1 ... 1 0 0]]


# Generate the public key

In [235]:
# The first 32 bytes of the public key come from the public seed
public_key = public_seed

# The columns of Q2 are concatenated to get a sequence of bits for the public key
Q2_bits = Q2.flatten('F')
remaining_bits = len(Q2_bits)
Q2_bytes = b''

while(remaining_bits>8):
  byte = ''
  for i in Q2_bits[:8]:
    # Reverse the order of bits
    byte = str(Q2_bits[i]) + byte
  # Turn the bits into a byte and concatenate
  Q2_bytes+=int(byte,2).to_bytes(1,byteorder='big')
  remaining_bits -= 8
  Q2_bits = Q2_bits[8:]

last_byte = ''
for bit in Q2_bits:
  # We reverse the order of the
  last_byte = str(bit) + last_byte

# Add the last byte
Q2_bytes+=int(last_byte,2).to_bytes(1,byteorder='big')
print(Q2_bytes)

b'\x00\xca\x00\xa6\xfff\xff\xff\x00\xea\xba\xd2>^\xff\x0e*\xe6\xc6\x00\xd6\xff\xff\x00\xff\xff\x00\x00\xff\xff\xff\x00:\x00^\x00\x00\xff\x00Z\x82j\x00\xa6\xae\xde\xffF\x96\x00\x96\xff\n~\xa6\xff\xff\xff\xa2\xff\xffN\x00Nr\x1e\xba\x00f\xff\x9an\xff\xe6\xff\xf2f\x00\x00\x92~\x82\xa2\x0e\xba\x92\xff\x00\x00\xbeN\x00\x92\x00\xff\x00\x00J\x9a\xff\x00B\xff\x00\xff\xaez\x00\x00\x0e\x1e\xff\x00.\x00nB\x02\xff\x00\xf6\xba\xff\x00.\x12:B\x06\xcan\xff\xdaj\x00\xff\xff\xa6\x82\x00&\xf2\x16\x00\xff\x00B\xe2\x16\x00\x0e\x00\x00\x00.\x02\xff\xff^\x00\x00\x02\x00\x00\x00\x00\x82\x00\xe2\xff\n\xffZ\x00\x00\xff\x00\xb2\xe6\xb2\x96\xff\xff\xae\xff\x00v\x00\x00\x1e\x92\xb6\x00\x1eV\xff\x00\xff\xff\xbe\x8a\x00"\xde\xff\x16\xffBj\x8e\xff\xea\xff\x00\x8e\x00\xff\x00\x00\x00\xa6\x9aVn\x00.\xf2\x00\xff\xb6nb*n\xd6\xff\xffr\x92\xff\x002N\x00\x00\xa2\x00^\x00\xa6\x00\x00\xff\xff\x00\xff\xff\x00n\xa2B\xb6\xff\xff\xe2v\x1e\xff\xea\xff\x16\xae\xff\xff\xda\x00\xff\xff\x00V\x92\x00>\x00f\xff\x00\xff\x8a~\xff\x00\xff\

In [236]:
# Add the bytes to the public key
public_key += Q2_bytes
print(public_key)
print(len(public_key))

b'\xf5kk\x85\x81\xabFa\xde\xc9\x88_\xf7\xa2@k\xf7*\xf7\xa8%\x96\xd9E\xb6\xae,\xb2b\xdd\xb7\xa2\x00\xca\x00\xa6\xfff\xff\xff\x00\xea\xba\xd2>^\xff\x0e*\xe6\xc6\x00\xd6\xff\xff\x00\xff\xff\x00\x00\xff\xff\xff\x00:\x00^\x00\x00\xff\x00Z\x82j\x00\xa6\xae\xde\xffF\x96\x00\x96\xff\n~\xa6\xff\xff\xff\xa2\xff\xffN\x00Nr\x1e\xba\x00f\xff\x9an\xff\xe6\xff\xf2f\x00\x00\x92~\x82\xa2\x0e\xba\x92\xff\x00\x00\xbeN\x00\x92\x00\xff\x00\x00J\x9a\xff\x00B\xff\x00\xff\xaez\x00\x00\x0e\x1e\xff\x00.\x00nB\x02\xff\x00\xf6\xba\xff\x00.\x12:B\x06\xcan\xff\xdaj\x00\xff\xff\xa6\x82\x00&\xf2\x16\x00\xff\x00B\xe2\x16\x00\x0e\x00\x00\x00.\x02\xff\xff^\x00\x00\x02\x00\x00\x00\x00\x82\x00\xe2\xff\n\xffZ\x00\x00\xff\x00\xb2\xe6\xb2\x96\xff\xff\xae\xff\x00v\x00\x00\x1e\x92\xb6\x00\x1eV\xff\x00\xff\xff\xbe\x8a\x00"\xde\xff\x16\xffBj\x8e\xff\xea\xff\x00\x8e\x00\xff\x00\x00\x00\xa6\x9aVn\x00.\xf2\x00\xff\xb6nb*n\xd6\xff\xffr\x92\xff\x002N\x00\x00\xa2\x00^\x00\xa6\x00\x00\xff\xff\x00\xff\xff\x00n\xa2B\xb6\xff\xff\xe2v\x1e\